In [ ]:
<a href="https://colab.research.google.com/github/annariha/StanCon-2024-BO-Stan/template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bayesian optimisation using Stan - StanCon 2024

### Setup

In [ ]:
# Use a repository of pre-built package binaries to speed-up installation
download.file("https://github.com/eddelbuettel/r2u/raw/master/inst/scripts/add_cranapt_jammy.sh",
              "add_cranapt_jammy.sh")
Sys.chmod("add_cranapt_jammy.sh", "0755")
system("./add_cranapt_jammy.sh")

# Install the R Packages we'll be using
install.packages(c("dplyr", "bayesplot", "cmdstanr"),
                  repos = c("https://stan-dev.r-universe.dev", getOption("repos")))


In this tutorial, we use the [cmdstanr](https://mc-stan.org/cmdstanr/articles/cmdstanr.html) R interface to CmdStan. We install and setup CmdStan as follows:

In [ ]:
# Install and setup CmdStan
download.file("https://github.com/stan-dev/cmdstan/releases/download/v2.35.0/colab-cmdstan-2.35.0.tgz",
              "cmdstan-2.35.0.tgz")
utils::untar("cmdstan-2.35.0.tgz")
cmdstanr::set_cmdstan_path("cmdstan-2.35.0")

Now, we load all required libraries: 

In [ ]:
library(cmdstanr)
library(here)
library(tidyverse)
library(khroma)